In [ ]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="demo_collection",
    dimension=384  # The vectors we will use in this demo has 384 dimensions
)

docs = [
    "Artificial intelligence was founded as an academic discipline in 1956.",
    "Alan Turing was the first person to conduct substantial research in AI.",
    "Born in Maida Vale, London, Turing was raised in southern England.",
]

vectors = [[ np.random.uniform(-1, 1) for _ in range(384) ] for _ in range(len(docs)) ]
data = [ {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"} for i in range(len(vectors)) ]
res = client.insert(
    collection_name="demo_collection",
    data=data
)

res = client.search(
    collection_name="demo_collection",
    data=[vectors[0]],
    filter="subject == 'history'",
    limit=2,
    output_fields=["text", "subject"],
)
print(res)

res = client.query(
    collection_name="demo_collection",
    filter="subject == 'history'",
    output_fields=["text", "subject"],
)
print(res)

res = client.delete(
    collection_name="demo_collection",
    filter="subject == 'history'",
)
print(res)

In [1]:
import boto3

In [ ]:
import boto3
import streamlit as st

st.subheader('RAG Using Knowledge Base from Amazon Bedrock', divider='rainbow')

if 'chat_history' not in st.session_state:
    st.session_state.chat_history = []

for message in st.session_state.chat_history:
    with st.chat_message(message['role']):
        st.markdown(message['text'])


bedrockClient = boto3.client('bedrock-agent-runtime', 'us-west-2')

def getAnswers(questions):
    knowledgeBaseResponse  = bedrockClient.invoke_agent(
        agentId='arn:aws:bedrock:us-west-2:905418480513:agent/WBEMY1OFQB',
        inputs={
            'questions': questions
        }
    )
    return knowledgeBaseResponse


questions = st.chat_input('Enter you questions here...')
if questions:
    with st.chat_message('user'):
        st.markdown(questions)
    st.session_state.chat_history.append({"role":'user', "text":questions})

    response = getAnswers(questions)
    # st.write(response)
    if 'observation' in response:
        observation = response['observation']

        # Check for final response text
        if 'finalResponse' in observation:
            text_output = observation['finalResponse'].get('text', '')

        # Check for action group output if final response isn't available
        elif 'actionGroupInvocationOutput' in observation:
            text_output = observation['actionGroupInvocationOutput'].get('text', '')

        else:
            text_output = "No text response found."

    with st.chat_message('assistant'):
        st.markdown(text_output)
    st.session_state.chat_history.append({"role":'assistant', "text": text_output})

    # if len(response['citations'][0]['retrievedReferences']) != 0:
    #     context = response['citations'][0]['retrievedReferences'][0]['content']['text']
    #     doc_url = response['citations'][0]['retrievedReferences'][0]['location']['s3Location']['uri']
        
    #     #Below lines are used to show the context and the document source for the latest Question Answer
    #     st.markdown(f"<span style='color:#FFDA33'>Context used: </span>{context}", unsafe_allow_html=True)
    #     st.markdown(f"<span style='color:#FFDA33'>Source Document: </span>{doc_url}", unsafe_allow_html=True)
    
    # else:
    #     st.markdown(f"<span style='color:red'>No Context</span>", unsafe_allow_html=True)
    